In [50]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
from financial_reports_collector import filings_collector
import datetime as dt
import requests
import pprint as pp

In [51]:
# Access page
ticker = input('ticker?').lower()
file_type = input('What file?(10-K,10-Q)').upper()

# Obtain HTML for search page
establishing_url = f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={ticker}&type={file_type}&dateb=&owner=exclude&start=0&count=100'
edgar_resp = requests.get(establishing_url)
edgar_str = edgar_resp.text

ticker?IEA
What file?(10-K,10-Q)10-k


In [52]:
# Find the document link and create df
reports_list = []
doc_link = ''
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        filing = cells[0].text
        doc_link = 'https://www.sec.gov' + cells[1].a['href']
        filing_date = dt.date.fromisoformat(cells[3].text)
        entry = {'filings' : filing, 'doc_link' : doc_link, 'date' : filing_date}
        reports_list.append(entry)

In [27]:
#  Find the XBRL link
for report in reports_list:
    doc_link = report['doc_link']
    doc_resp = requests.get(doc_link)
    doc_str = doc_resp.text
    soup = BeautifulSoup(doc_str, 'html.parser')
    try:
        table_tag = soup.find('table', class_='tableFile', summary='Data Files')
        rows = table_tag.find_all('tr')
        time.sleep(.01)
        for row in rows:
            cells = row.find_all('td')
            if len(cells) > 3:
                if 'INS' in cells[3].text:
                    xbrl_link = 'https://www.sec.gov' + cells[2].a['href']
                    report.update({'xbrl_link' : xbrl_link})
    except:
        report.update({'xbrl_link' : 'N/A'})

1
2
3
4
5
6
7
8
9
10


In [56]:
filings_df = pd.DataFrame(reports_list)
filings_df.tail()
# print(reports_list[0])

,date,doc_link,filings
0,2018-03-06,https://www.sec.gov/Archives/edgar/data/165236...,10-K
1,2017-03-30,https://www.sec.gov/Archives/edgar/data/165236...,10-K


In [39]:
tag_name_list = []

lxml_ten_q = 'https://www.sec.gov/Archives/edgar/data/1326380/000132638018000122/gme-20181103.xml'

xbrl_resp = requests.get(lxml_ten_q)
xbrl_str = xbrl_resp.text

# Find and print stockholder's equity
soup = BeautifulSoup(xbrl_str, 'lxml')
tag_list = soup.find_all()

        



In [40]:
for tag in tag_list:
    tag_name_list.append(tag.name)
#     if tag.name == 'us-gaap:grossprofit':
#         print(int(tag.text))

In [49]:
pprinted = pp.PrettyPrinter(indent=4)
pprinted.pprint(tag_name_list)

[   'html',
    'body',
    'xbrli:xbrl',
    'link:schemaref',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:startdate',
    'xbrli:enddate',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:startdate',
    'xbrli:enddate',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:startdate',
    'xbrli:enddate',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:period',
    'xbrli:startdate',
    'xbrli:enddat

    'xbrldi:explicitmember',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:segment',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:segment',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:segment',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrli:segment',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrldi:explicitmember',
    'xbrli:period',
    'xbrli:instant',
    'xbrli:context',
    'xbrli:entity',
    'xbrli:identifier',
    'xbrl

    'us-gaap:deferredincometaxexpensebenefit',
    'us-gaap:deferredincometaxexpensebenefit',
    'us-gaap:deferredtaxassetsliabilitiesnetnoncurrent',
    'us-gaap:deferredtaxassetsliabilitiesnetnoncurrent',
    'us-gaap:deferredtaxassetsliabilitiesnetnoncurrent',
    'us-gaap:deferredtaxassetsliabilitiesnetnoncurrent',
    'us-gaap:deferredtaxassetsliabilitiesnetnoncurrent',
    'us-gaap:deferredtaxliabilitiesnoncurrent',
    'us-gaap:deferredtaxliabilitiesnoncurrent',
    'us-gaap:deferredtaxliabilitiesnoncurrent',
    'us-gaap:depreciationandamortization',
    'us-gaap:depreciationandamortization',
    'us-gaap:depreciationandamortization',
    'us-gaap:depreciationandamortization',
    'us-gaap:derivativegainlossonderivativenet',
    'us-gaap:derivativegainlossonderivativenet',
    'us-gaap:derivativegainlossonderivativenet',
    'us-gaap:derivativegainlossonderivativenet',
    'us-gaap:dividends',
    'us-gaap:dividends',
    'us-gaap:dividends',
    'us-gaap:dividends',
    'us-g

In [ ]:
'dei:currentfiscalyearenddate'
'dei:entitycommonstocksharesoutstanding'
'dei:entityfilercategory'
'dei:entityregistrantname'
'dei:tradingsymbol'